In [1]:
# permite a utilização de caminhos relativos ao projeto, mesmo no diretório de notebooks

from knapsax.utils import setrootdir

setrootdir("knapsax")

'Directory knapsax successfully loaded as current working directory.'

In [33]:
from typing import List, Tuple, Dict, Callable
import random

import numpy as np

from knapsax.optimization import Problem, Maximize, Solution, Knapsack

In [3]:
knapsack = Knapsack(instance_file="data/knapsack-instance.txt")
knapsack

Knapsack(file=data/knapsack-instance.txt, n_items=100, capacity=1550)

In [139]:
class HCSolution(Solution):
    def __init__(self, knapsack: Knapsack):
        super().__init__(knapsack)

    def generate_solution(self):

        self.shuffled_items = self.knapsack.items.copy()
        random.shuffle(self.knapsack.items)

        while self.total_weight <= self.knapsack.capacity and len(self.shuffled_items) > 0:
            item = self.shuffled_items.pop(0)
            if self.total_weight + item.weight <= self.knapsack.capacity:
                self.x.append(item)
                self.total_value += item.value
                self.total_weight += item.weight
                self.n_items += 1
                random.shuffle(self.shuffled_items)
            else:
                self.shuffled_items.append(item)
                break
    
    def move(self):

        index_out = random.randint(0, len(self.x) - 1)
        item_out = self.x.pop(index_out)
        self.total_weight -= item_out.weight
        self.total_value -= item_out.value
        self.n_items -= 1
        self.shuffled_items.append(item_out)
        random.shuffle(self.shuffled_items)

        while self.total_weight <= self.knapsack.capacity and len(self.shuffled_items) > 0:
            item = self.shuffled_items[0]
            if self.total_weight + item.weight <= self.knapsack.capacity:
                self.shuffled_items.pop(0)
                self.x.append(item)
                self.total_value += item.value
                self.total_weight += item.weight
                self.n_items += 1
                random.shuffle(self.shuffled_items)
            else:
                self.shuffled_items.append(item)
                break

In [140]:
class HillClimbing:
    def __init__(self, problem: Problem, knapsack: Knapsack, max_iterations: int = 1000):
        self.problem = problem
        self.knapsack = knapsack
        self.max_iterations = max_iterations
        self.n_iterations = 0
        self._called = False
        
        self.initial_solution = self.best_solution = HCSolution(knapsack)
        self.best_cost = self.best_solution.total_value
        self.history = {
            "best_solution": [],
            "best_cost": [],
        }

    def run(self):

        if self._called:
            raise RuntimeError("This method can only be called once.")

        current_solution = self.initial_solution

        while self.n_iterations < self.max_iterations:

            current_solution.move()
            new_cost = current_solution.total_value

            if self.problem(new_cost, self.best_cost):
                self.best_solution = current_solution
                self.best_cost = new_cost

            self.history["best_solution"].append(self.best_solution)
            self.history["best_cost"].append(self.best_cost)
                
            self.n_iterations += 1
        
        self._called = True

    def __str__(self):
        return f"Iteration: {self.n_iterations}\n\nBest cost: {self.best_cost}\n\nBest solution weight: {self.best_solution.total_weight}\n\nBest solution: {self.best_solution.x}"
    
    def __repr__(self):
        return f"Iteration: {self.n_iterations}\n\nBest cost: {self.best_cost}\n\nBest solution weight: {self.best_solution.total_weight}\n\nBest solution: {self.best_solution.x}"

In [141]:
hc = HillClimbing(
    problem=Maximize(),
    knapsack=knapsack,
    max_iterations=20_000,
)

hc.run()

hc

Iteration: 20000

Best cost: 2022

Best solution weight: 1540

Best solution: [Item(value=76, weight=65), Item(value=80, weight=67), Item(value=80, weight=69), Item(value=49, weight=34), Item(value=85, weight=74), Item(value=88, weight=76), Item(value=72, weight=60), Item(value=92, weight=78), Item(value=65, weight=50), Item(value=61, weight=49), Item(value=78, weight=64), Item(value=80, weight=67), Item(value=94, weight=80), Item(value=92, weight=78), Item(value=69, weight=53), Item(value=94, weight=80), Item(value=81, weight=72), Item(value=62, weight=48), Item(value=95, weight=81), Item(value=92, weight=78), Item(value=94, weight=80), Item(value=70, weight=59), Item(value=92, weight=78)]